In [161]:
# -*- coding: utf-8 -*-


import numpy as np
from bs4 import BeautifulSoup
import requests
import csv

HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.75'}
COOKIES = {'__cfduid': 'd44db92eab1474bde64206da8c5e0f5b11507046474',
           'cf_clearance': '187c74875d4cb11dd9c985baae1cb43c272bf631-1507209348-1800',
           'language': 'en-US',
           'cdmu': '1507082693368',
           'cdmblk2': '0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0; cdmblk=0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0',
           'cdmtlk': '0:0:0:0:0:0:0:0:0:0:0:0:0:0',
           'cdmgeo': 'us',
           'cdmbaserate': '2.1',
           'cdmbaseraterow': '1.1',
           'cdmint': '0'}


def get_soup(url, headers=HEADERS, cookies=COOKIES):
    req = requests.get(url, headers=headers, cookies=cookies)
    html = req.text
    soup = BeautifulSoup(html, 'lxml')
    return soup

# get player info from the page
# for duo or squad data, change the string at line 47
def get_player(soup):
    url = 'https://pubg.me'
    players = (soup.body.div.next_sibling.div.div.next_sibling.div.div.next_sibling
              .table.tbody.find_all('tr'))
    link_pool = []
    for player in players:
        link = url + player.find_all(class_="sidebar-user-link")[0].attrs['href'] + '/solo'
        link_pool.append(link)
    return link_pool

# get player data from each page
def get_data(url):
    soup = get_soup(url)
    data = soup.find_all('div', class_='card mb-3')
    ID = soup.find_all('div', class_='card-header')[0].contents[0].contents[1].contents[0].string
    data_list = []
    for i in data:
        data_list += i.find_all('div', class_='col-md-4')
    value_list = []
    label_list = []
    label_list.append("ID")
    value_list.append(ID)
    for i in data_list:
        for j in i.contents:
            value_list.append(j.contents[0].string)
            label_list.append(j.contents[-1].string)
    return [label_list, value_list]

# concatenate all the data
def get_all(link_pool):
    i=0
    while(get_data(link_pool[i])[0] == ['ID']):
        i = i+1
    data_list = np.array(get_data(link_pool[i])[0])
    for link in link_pool:
        user = get_data(link)[1]
        if len(user) == 1:                          
            data_list = np.vstack([data_list, user+["NAN"]*37])
        else:
            data_list = np.vstack([data_list, user])
    return data_list

#output the data 
def out_csv(data, place):
    name = 'pubg_%s.csv' % place
    with open(name, "w") as f:
        writer = csv.writer(f)
        writer.writerows(data)

if __name__ == '__main__':
    data_as = get_all(get_player(get_soup('https://pubg.me/players/rating/?season=2017-pre5&region=as&match=solo')))
    out_csv(data_as, 'as')
    
    data_eu = get_all(get_player(get_soup('https://pubg.me/players/rating/?season=2017-pre5&region=eu&match=solo')))
    out_csv(data_eu, 'eu')

    data_na = get_all(get_player(get_soup('https://pubg.me/players/rating/?season=2017-pre5&region=na&match=solo')))
    out_csv(data_na, 'na')

In [1]:
import pandas as pd
def get_player(name):
    url = 'https://pubg.me/player/'
    link_pool = []
    for player in name:
        link = url + player + '/duo'
        link_pool.append(link)
    return link_pool

# concatenate all the data
def get_all(link_pool):
    i=0
    while(get_data(link_pool[i])[0] == ['ID']):
        i = i+1
    data_list = np.array(get_data(link_pool[i])[0])
    for link in link_pool:
        user = get_data(link)[1]
        if len(user) == 1:                          
            data_list = np.vstack([data_list, user+["NAN"]*37])
        else:
            data_list = np.vstack([data_list, user])
    return data_list


# get player data from each page
def get_data(url):
    soup = get_soup(url)
    data = soup.find_all('div', class_='card mb-3')
    ID = url.split("/")[4]
    if data == []:
        return [["ID"] + ["NAN"]*37 , [ID] + ["NAN"]*37]
    else:
        data_list = []
        for i in data:
            data_list += i.find_all('div', class_='col-md-4')
        value_list = []
        label_list = []
        label_list.append("ID")
        value_list.append(ID)
        for i in data_list:
            for j in i.contents:
                value_list.append(j.contents[0].string)
                label_list.append(j.contents[-1].string)
        return [label_list, value_list]


In [2]:
import cfscrape
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv

HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
COOKIES = {'__cfduid': 'd44db92eab1474bde64206da8c5e0f5b11507046474',
           'cf_clearance': '187c74875d4cb11dd9c985baae1cb43c272bf631-1507209348-1800',
           'language': 'en-US',
           'cdmu': '1507082693368',
           'cdmblk2': '0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0; cdmblk=0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0,0:0:0:0:0:0:0:0:0:0:0:0:0:0',
           'cdmtlk': '0:0:0:0:0:0:0:0:0:0:0:0:0:0',
           'cdmgeo': 'us',
           'cdmbaserate': '2.1',
           'cdmbaseraterow': '1.1',
           'cdmint': '0'}


def get_soup(url):
    scraper = cfscrape.create_scraper()
    req = scraper.get(url)
    html = req.text
    soup = BeautifulSoup(html, 'lxml')
    return soup

In [3]:
data = pd.read_csv("PUBG_Player_Statistics.csv")
name = data['player_name']

In [4]:
name = name.tolist()

In [5]:
pool = get_player(name)

In [12]:
def out_csv(pool):
    data = get_all(pool)
    name = 'pubg_data_duo.csv' 
    with open(name, "w") as f:
        writer = csv.writer(f)
        writer.writerows(data)


In [ ]:
out_csv(pool[:10000])